# Gutenberg N-Grams

Start the Spark Context: 

In [1]:
import sys, os
os.environ['SPARK_HOME'] = '/Users/rok/spark'
os.environ['SPARK_EXECUTOR_MEMORY'] = '2g'
sys.path.insert(0, '/Users/rok/spark/python/')
sys.path.insert(0, '/Users/rok/spark/python/lib/py4j-0.8.2.1-src.zip')

import pyspark

In [2]:
conf = pyspark.SparkConf()

In [3]:
conf.set('spark.executor.memory', '2g')
conf.set('spark.driver.memory', '4g')

In [4]:
sc = pyspark.SparkContext(master = "local[4]", conf = conf)

## Make a key-value RDD of book metadata and text

Getting data into spark from a collection of local files is a very common task. A useful pattern to keep in mind is the following: 

1. make a list of filenames and distribute it among the workers
3. "map" each filename to the data you want to get out
4. now you are left with the RDD of raw data distributed among the workers!

The [`gutenberg_cleanup`](gutenberg_cleanup.py) module contains two functions that can help with this: `get_text` and `get_metadata`.

They pretty much do the obvious: 

`get_metadata` returns a metadata object with various useful fields that will be used to create a unique key for each book

`get_text` returns the raw text extracted from HTML, cleaned of tags and punctuation and converted to lower case. 

### 1. Distributing the filenames

In [127]:
import glob
flist = glob.glob('/Users/rok/python_src/gutenberg/dl-cache/*html')
print 'number of books: ', len(flist)

number of books:  5022


In [6]:
files_rdd = sc.parallelize(flist)

In [7]:
files_rdd.take(5)

['/Users/rok/python_src/gutenberg/dl-cache/1000.html',
 '/Users/rok/python_src/gutenberg/dl-cache/1001.html',
 '/Users/rok/python_src/gutenberg/dl-cache/1002.html',
 '/Users/rok/python_src/gutenberg/dl-cache/1003.html',
 '/Users/rok/python_src/gutenberg/dl-cache/1004.html']

### 2. Map the filenames to metadata, text key-value pairs

Use the `get_text` and `get_metadata` functions to construct a key,value pair RDD, where `key` is the dictionary returned by `get_metadata`. For the `value` of each `key`,`value` pair use the raw text returned by `get_text`. 

Hint: in Python, there are many ways to make a string, but a pretty easy one is like this: 

    "bla_%s"%(var)"

where `var` matches the `%s` and is a variable that can be converted to a string. You can include more `%s` (or `%d`, `%f` etc) and more variables in the tuple that follows. 

In [116]:
import gutenberg_cleanup
reload(gutenberg_cleanup)
from gutenberg_cleanup import get_metadata, get_text, get_gid

In [22]:
text_rdd = (files_rdd.map(lambda filename: 
                         (get_metadata(get_gid(filename)), get_text(filename))))

So that we don't have to constantly re-load the data off disk, lets cache this RDD: 

In [23]:
# text_rdd.cache()

Take a look at the first set of keys:

In [24]:
text_rdd.keys().take(50)

[{'birth_year': None,
  'death_year': None,
  'first_name': None,
  'gid': 1000,
  'last_name': None,
  'title': '- No Title -'},
 {'birth_year': u'1265',
  'death_year': u'1321',
  'first_name': None,
  'gid': 1001,
  'last_name': u'Dante Alighieri',
  'title': u"Divine Comedy, Longfellow's Translation, Hell"},
 {'birth_year': u'1807',
  'death_year': u'1882',
  'first_name': None,
  'gid': 1002,
  'last_name': u'Dante Alighieri',
  'title': u"Divine Comedy, Longfellow's Translation, Purgatory"},
 {'birth_year': u'1265',
  'death_year': u'1321',
  'first_name': None,
  'gid': 1003,
  'last_name': u'Dante Alighieri',
  'title': u"Divine Comedy, Longfellow's Translation, Paradise"},
 {'birth_year': u'1807',
  'death_year': u'1882',
  'first_name': None,
  'gid': 1004,
  'last_name': u'Dante Alighieri',
  'title': u"Divine Comedy, Longfellow's Translation, Complete"},
 {'birth_year': u'1772',
  'death_year': u'1844',
  'first_name': None,
  'gid': 1005,
  'last_name': u'Dante Alighieri',

If you look at just the first few entries it becomes clear that we're going to have to do some quality control here. For example, we probably don't want books with "None" as either of the author names, and likewise we have to have the birth date in order to be able to create a time series out of the data in the end. 

Construct an RDD, as above, except that you filter out all elements that lack a value for `title`, `first_name`, `last_name`, or `birth_year`.

In [27]:
filtered_rdd = (files_rdd.map(lambda filename: (filename, get_metadata(get_gid(filename))))
                         .filter(lambda (filename,meta): all([meta[name] is not None for name in ['title', 'first_name', 'last_name', 'birth_year']])))

In [28]:
filtered_rdd.take(10)

[('/Users/rok/python_src/gutenberg/dl-cache/101.html',
  {'birth_year': u'1954',
   'death_year': None,
   'first_name': u'Bruce',
   'gid': 101,
   'last_name': u'Sterling',
   'title': u'The Hacker Crackdown: Law and Disorder on the Electronic Frontier'}),
 ('/Users/rok/python_src/gutenberg/dl-cache/1013.html',
  {'birth_year': u'1866',
   'death_year': u'1946',
   'first_name': u'H. G. (Herbert George)',
   'gid': 1013,
   'last_name': u'Wells',
   'title': u'The First Men in the Moon'}),
 ('/Users/rok/python_src/gutenberg/dl-cache/1014.html',
  {'birth_year': u'1874',
   'death_year': u'1940',
   'first_name': u'B. M.',
   'gid': 1014,
   'last_name': u'Bower',
   'title': u'The Lure of the Dim Trails'}),
 ('/Users/rok/python_src/gutenberg/dl-cache/1015.html',
  {'birth_year': u'1823',
   'death_year': u'1893',
   'first_name': u'Francis',
   'gid': 1015,
   'last_name': u'Parkman',
   'title': u'The Oregon Trail: Sketches of Prairie and Rocky-Mountain Life'}),
 ('/Users/rok/python

How many do we have left? 

In [207]:
print 'number of books after filtering: ', filtered_rdd.count()

number of books after filtering:  3247


Some of the books end up in multiple files, but they should all have the same gid. 

To check for this we will use one of the most basic and common Map/Reduce patterns: 

* map the data into `key`,`value` pairs where `key` is the quantity we want to count and `value` is just 1. 
* invoke a reduction *by key*, where the reduction operator is a simple addition

Finally, we will sort the result and print out the first few elements to check whether we have to worry about documents spanning multiple files or not. 

The RDD operations that are needed are [`reduceByKey`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) and [sortBy](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortBy).

For the `keyFunc` of the call to `sortBy`, use a `lambda` function that extracts the counts obtained from the `reduceByKey`. 

In [29]:
from operator import add

In [30]:
(filtered_rdd.map(lambda (filename, meta): (meta['gid'], 1))
             .reduceByKey(add)
             .sortBy(lambda (key,count): count, False)
             .take(20))

[(3772, 40),
 (3332, 33),
 (3425, 23),
 (2440, 16),
 (4022, 5),
 (1079, 2),
 (2048, 1),
 (3072, 1),
 (4228, 1),
 (68, 1),
 (2056, 1),
 (4780, 1),
 (12, 1),
 (4092, 1),
 (2064, 1),
 (3416, 1),
 (20, 1),
 (5388, 1),
 (2072, 1),
 (3076, 1)]

Looks like we have a few that are made up of multiple sections. To combine them, we will use `groupByKey` (see warning below) which will result in having an RDD of `gid`'s as keys and lists of filenames for each `gid`. Once we have the lists combined, we will do another `map` to read the actual text. Note that doing this is much cheaper than reading the text first and then combining (if we had used the `text_rdd` from the top of the notebook, for example) because we are only sending around file names rather than all the document data. 

** warning about `groupByKey` **: this is in general the most expensive of the grouping/accumulating operations because it sends the entire RDD across the network in order to group the values in a list. No local aggregation happens on the partitions. By contrast, `reduceByKey` and `aggregateByKey` do the reductions/aggregations *localy* on each partition first and only then shuffle the data around, resulting in much less traffic. However, since we actually aren't sending much data here and since we only have a few elements that need to be grouped, it's fine to use it for simplicity's sake. In general, however, `groupByKey` likes to cause memory problems and runs very slowly. 

So, the procedure for doing this final step of pre-processing: 

1. `map` the `filtered_rdd` to have the `gid` as a key and `filename` as the value
2. do `groupByKey`
3. `map` the resulting (`gid`,`flist`) key value pair RDD to yield a (`gid`, `<combined text>`) RDD

*hint*: the `join` function in the `string` module can combine a list of strings efficiently. 
*hint2*: `groupByKey` will give you an *iterable* of filenames as a result -- you can use it inside a list comprehension with the `get_text` function 

In [76]:
cleaned_rdd = (filtered_rdd.map(lambda (filename, meta): (meta['gid'], filename))
                           .groupByKey()
                           .map(lambda (gid,flist): (gid,string.join([get_text(f) for f in flist]))))

## Processing the data

We're finished with the pre-processing. `cleaned_rdd` contains `gid`'s as keys and text as values. If we want some other piece of metadata, we can just call the `get_metadata` function inside a `map` to extract it. 

### Histogram of book years
Now we're ready to start asking some questions of the data. To begin with, lets do a simple histogram of the year distribution of the books. Since we don't have original publication dates, we just use the simple formula: 

$year = max\left((year_{birth} + year_{death})/2, year_{birth} + 30\right)$. 

This means that for authors that are still living, we assume they wrote their book at 30.  

In [118]:
def publication_year(meta) : 
    birth_year = int(meta['birth_year'])
    if meta['death_year'] is None : 
        return int(meta['birth_year']) + 30
    else :
        death_year = int(meta['death_year'])
        return max((birth_year + death_year) / 2.0, birth_year+30)

In [125]:
year_rdd = cleaned_rdd.map(lambda (gid, text): publication_year(get_metadata(gid)))

The histogram function actually already exists in the Spark API (but it didn't use to!). However, for fun we will write our own. Calculating the histogram can be split up into two parts. First, we need to figure out which bin each value corresponds to: 

1. take bins and a value as input
2. calculate the bin that the value maps to and return (`bin`, 1) pair

Second, we need to do a simple `reduceByKey` where we just add up all the values belonging to each bin. 

In [132]:
from bisect import bisect_right
import numpy as np
def get_bin(bin_edges, value) : 
    return bisect_right(bin_edges, value) - 1

In [136]:
def histogram(rdd, nbins = 100, min_val=None, max_val=None) :
    # if either min_val or max_val are missing, get them from the data
    if min_val is None : 
        min_val = rdd.min()
    if max_val is None : 
        max_val = rdd.max()
        
    bin_edges = np.linspace(min_val,max_val,nbins+1)
    
    binned_rdd = rdd.map(lambda x: get_bin(bin_edges, x))
    
    res = binned_rdd.countByValue()
    
    return res, .5*(bin_edges[:-1]+bin_edges[1:])

In [134]:
res = histogram(year_rdd)

In [135]:
res

defaultdict(<type 'int'>, {0: 1, 3: 1, 15: 1, 16: 1, 42: 3, 44: 1, 67: 1, 70: 2, 73: 2, 77: 6, 78: 127, 79: 1, 80: 3, 81: 12, 82: 74, 83: 11, 84: 13, 85: 23, 86: 43, 87: 39, 88: 40, 89: 69, 90: 98, 91: 443, 92: 432, 93: 811, 94: 707, 95: 508, 96: 43, 97: 18, 98: 9, 99: 8, 100: 1})

In [ ]:
# First figure out which bin each document falls into: 
    binned = rdd.map(lambda (_,x): get_bins(len(x),binedges))

    # Then add up all the bins: 
    res = binned.countByValue()

    # This is a sparse result -- turn into a dense vector for plotting: 
    res_full = np.zeros(nbins)
    overflow = 0
    for item in res.iteritems() : 
        if item[0] > len(res_full)-1 : overflow += item[1]
        else: res_full[item[0]] = item[1]
    res_full[-1] += overflow
    
    return bins, res_full
